In [1]:
#   Détecteurr d'Intrusion Réseau par IA
#   
#   Système de détection d'intrusion 
#   basé sur le Machine Learning (Random Forest)
#   
#   Dataset : trafic réseau réel capturé en laboratoire virtuel
#   Modèle  : Random Forest (100 arbres)
#   Classes : normal / nmap / dos

print("Détecteur d'Intrusion Réseau par IA")
print("   Chargement de l'environnement...")

Détecteur d'Intrusion Réseau par IA
   Chargement de l'environnement...


In [2]:
# Installation des bibliothèques
import subprocess
subprocess.run(["pip", "install", "pandas", 
                "scikit-learn", "matplotlib", 
                "seaborn"], capture_output=True)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("Environnement prêt")

Environnement prêt


In [ ]:
# Chargement du dataset depuis GitHub
URL = "https://raw.githubusercontent.com/abderemaneattoumani/ids-reseau-ia/main/Datasets/dataset_sample.csv"

df = pd.read_csv(URL)

print(f"Dataset chargé depuis GitHub")
print(f"   Lignes   : {len(df):,}")
print(f"   Colonnes : {list(df.columns)}")
print(f"\nRépartition :")
print(df['label'].value_counts())

In [ ]:
# Encodage
le_protocol = LabelEncoder()
le_src = LabelEncoder()
le_dst = LabelEncoder()
le_label = LabelEncoder()

df_model = df.copy()
df_model['protocol'] = le_protocol.fit_transform(df_model['protocol'].astype(str))
df_model['ip_src']   = le_src.fit_transform(df_model['ip_src'].astype(str))
df_model['ip_dst']   = le_dst.fit_transform(df_model['ip_dst'].astype(str))
df_model['label_encoded'] = le_label.fit_transform(df_model['label'])

features = ['protocol', 'ip_src', 'ip_dst',
            'port_src', 'port_dst',
            'packet_length', 'ttl']

X = df_model[features]
y = df_model['label_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Entraînement
print("Entraînement du Random Forest...")
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Modèle entraîné — Précision : {score*100:.2f}%")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Détecteur d\'Intrusion Réseau — Résultats', 
             fontsize=16, fontweight='bold')

# Matrice de confusion
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_label.classes_,
            yticklabels=le_label.classes_, ax=axes[0])
axes[0].set_title('Matrice de Confusion')
axes[0].set_ylabel('Réalité')
axes[0].set_xlabel('Prédiction')

# Importance des features
importances = pd.Series(
    model.feature_importances_, index=features
).sort_values(ascending=True)
importances.plot(kind='barh', ax=axes[1], color='steelblue')
axes[1].set_title('Importance des Features')

plt.tight_layout()
plt.show()
print("Visualisations générées")

In [ ]:
def tester_paquet(description, protocol, ip_src, ip_dst,
                   port_src, port_dst, packet_length, ttl):

    try:
        proto = le_protocol.transform([str(protocol)])[0]
    except:
        proto = 0
    try:
        src = le_src.transform([str(ip_src)])[0]
    except:
        src = 0
    try:
        dst = le_dst.transform([str(ip_dst)])[0]
    except:
        dst = 0

    paquet = [[proto, src, dst, port_src, port_dst, packet_length, ttl]]
    pred = model.predict(paquet)[0]
    label = le_label.inverse_transform([pred])[0]
    probas = model.predict_proba(paquet)[0]
    confiance = max(probas) * 100

    print(f"\n{'='*50}")
    print(f"TEST : {description}")
    print(f"{'='*50}")
    print(f"VERDICT    : {label.upper()}")
    print(f"   Confiance : {confiance:.1f}%")
    for classe, proba in zip(le_label.classes_, probas):
        barre = "█" * int(proba * 20)
        print(f"   {classe:<8} : {barre:<20} {proba*100:.1f}%")

# Les trois tests
tester_paquet("Connexion SSH normale",
    "TCP", "192.168.100.20", "192.168.100.10",
    52341, 22, 512, 64)

tester_paquet("Scan Nmap",
    "TCP", "192.168.100.20", "192.168.100.10",
    44321, 445, 44, 45)

tester_paquet("Attaque DoS SYN Flood",
    "TCP", "192.168.100.20", "192.168.100.10",
    2309, 0, 60, 64)